# Writing CUDA Kernels
-------------------
## Contents
- Example - computing pairwise distances given a dataset of points
    - Solution using serial for-loops
    - Simple CUDA solution
    - Optimized CUDA solution
- Example - matrix multiplication
    - Simple CUDA solution
    - Optimized CUDA solution
-------------------

Let's see the GPU allocated to us using the ```nvidia-smi``` shell command.

In [ ]:
!nvidia-smi

In [ ]:
from numba import cuda
cuda.detect()

In [ ]:
from numba import njit, cuda, vectorize, guvectorize, stencil
from numba import prange
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve as scipy_convolve

%matplotlib inline

In [ ]:
# from numba.core.errors import NumbaPerformanceWarning
# import warnings
# warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

---------------------
# [Section 1] Example - Computing Pairwise Distances

This computation shows up in many practical scenarios/problems:
- Distance matrices for clustering algorithms
- k-Nearest Neighbors (kNN) classification algorithm
- Estimation of topological manifolds
- Similarity-based searches such as recommendations or information retrieval

![](./distance_matrix.jpg)

Notice that:
- The cells of the distance matrix can be filled in parallel since the distance computation (Euclidean distance, for example) has no external dependencies
- Typically, the distance between A and B is the same as B and A. Let's ignore this property for now.

In [ ]:
n_samples = 1000
xy_coordinates = np.random.rand(n_samples, 2)

In [ ]:
xy_coordinates

---------------------
# [Section 2] Serial Solution using For-loops

In [ ]:
from scipy.spatial import distance

def distance_matrix_serial(data):
    distance_matrix = np.zeros((data.shape[0], data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(data.shape[0]):
            distance_matrix[i, j] = distance.euclidean(data[i], data[j])
    return distance_matrix

In [ ]:
import time

start_time = time.time()
print("--- [Serial] Starting the timer ---")
result_serial = distance_matrix_serial(xy_coordinates)
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

---------------------
# [Section 3] Simple CUDA Solution

In [ ]:
from numba import cuda
from scipy.spatial import distance

# one block computes one cell inside the distance matrix
# single thread inside the block performs the computation

@cuda.jit
def distance_matrix_cuda(data, distance_matrix):

    i, j = cuda.grid(2)

    if i > len(data) or j > len(data):
        return
    
    distance_matrix[i, j] = ((data[i,0] - data[j,0])**2 + (data[i,1] - data[j,1])**2)**0.5

    return

In [ ]:
import time

cuda.select_device(2)  # Change to the device ID you want to use

num_blocks = (n_samples, n_samples)
num_threads_per_block = 1

result_cuda = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()

start_time = time.time()
print("\n--- [simple CUDA] Starting the timer ---")
result_cuda = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

print("\nYour result is {}".format("correct!" if np.allclose(result_serial, result_cuda) else "incorrect."))

---------------------
# [Section 4] Simple CUDA Solution + Shared CUDA Memory

In [ ]:
# one block computes distances from current point to all other points
# each point is handled by a separate thread inside the block
# threads inside the block take the current point from shared memory of the block

from numba import cuda, float64

@cuda.jit
def distance_matrix_cuda_optimized(data, distance_matrix):

    current_point = cuda.shared.array(shape=(2), dtype=float64)

    i = cuda.blockIdx.x
    if i < data.shape[0]:
        current_point = data[i]
    
    cuda.syncthreads()

    j = cuda.threadIdx.x
    if j < data.shape[0]:
        distance_matrix[i, j] = ((current_point[0] - data[j,0])**2 + (current_point[1] - data[j,1])**2) ** 0.5

    return

In [ ]:
import time

num_blocks = n_samples
num_threads_per_block = n_samples

result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()

start_time = time.time()
print("\n--- [simple CUDA + shared memory] Starting the timer ---")
result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))

print("\nYour result is {}".format("correct!" if np.allclose(result_serial, result_cuda_optimized) else "incorrect."))

### Seeing the difference with and without shared memory more clearly (100 repeats)...

In [ ]:
def repeat_simple_cuda(n_samples=1000, n_repeats=100):
    exec_times = []
    for _ in range(n_repeats):
        xy_coordinates = np.random.rand(n_samples, 2)
        num_blocks = (n_samples, n_samples)
        num_threads_per_block = 1
        result_cuda = np.zeros((n_samples, n_samples), dtype=float)
        start_time = time.time()
        distance_matrix_cuda[num_blocks, num_threads_per_block](xy_coordinates, result_cuda); cuda.synchronize()
        exec_times.append(time.time() - start_time)
    return exec_times

def repeat_optimized_cuda(n_samples=1000, n_repeats=100):
    exec_times = []
    for _ in range(n_repeats):
        xy_coordinates = np.random.rand(n_samples, 2)
        num_blocks = n_samples
        num_threads_per_block = n_samples
        result_cuda_optimized = np.zeros((n_samples, n_samples), dtype=float)
        start_time = time.time()
        distance_matrix_cuda_optimized[num_blocks, num_threads_per_block](xy_coordinates, result_cuda_optimized); cuda.synchronize()
        exec_times.append(time.time() - start_time)
    return exec_times

In [ ]:
exec_times_simple_cuda = repeat_simple_cuda()
exec_times_optimized_cuda = repeat_optimized_cuda()

print(f"Simple CUDA (100 repeats): {np.mean(exec_times_simple_cuda)} ({np.std(exec_times_simple_cuda)})")
print(f"Optimized CUDA (100 repeats): {np.mean(exec_times_optimized_cuda)} ({np.std(exec_times_optimized_cuda)})")

---------------------
# [Section 5] Matrix multiplication - Simple CUDA Solution

In [ ]:
# each thread fills in one cell of the output matrix C

@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]): # or range(B.shape[1])
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [ ]:
import numpy as np
import math

TPB = 16

# Initialize the data arrays
A = np.random.rand(TPB*100, TPB*100) # random matrix
B = np.random.rand(TPB*100, TPB*100) # random matrix
C = np.zeros((A.shape[0], B.shape[1])) # output matrix

# Configure the blocks
num_threads_per_block = 1
num_blocks = (C.shape[0], C.shape[1])
print(f"num_blocks: {num_blocks}, num_threads_per_block: {num_threads_per_block}")

# JIT compilation/caching
matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()

# --------------------------
C = np.zeros((A.shape[0], B.shape[1])) # output matrix
start_time = time.time()
print("\n--- [simple CUDA] Starting the timer ---")
# Start the kernel 
matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))
# --------------------------

# Check result with serial/CPU version
result_serial = np.matmul(A, B)
print("\nYour result is {}".format("correct!" if np.allclose(result_serial, C) else "incorrect."))

### Matrix multiplication - Simple CUDA Solution + Different Blocks/Threads Configuration (same performance as using shared memory!)

In [ ]:
import numpy as np
import math

TPB = 16

# Initialize the data arrays
A = np.random.rand(TPB*100, TPB*100) # random matrix
B = np.random.rand(TPB*100, TPB*100) # random matrix
C = np.zeros((A.shape[0], B.shape[1])) # output matrix

# Configure the blocks
num_threads_per_block = (TPB, TPB)
blocks_x = int(math.ceil(A.shape[0] / num_threads_per_block[0]))
blocks_y = int(math.ceil(B.shape[1] / num_threads_per_block[1]))
num_blocks = (blocks_x, blocks_y)
print(f"num_blocks: {num_blocks}, num_threads_per_block: {num_threads_per_block}")

# JIT compilation/caching
matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()

# --------------------------
C = np.zeros((A.shape[0], B.shape[1])) # output matrix
start_time = time.time()
print("\n--- [simple CUDA] Starting the timer ---")
# Start the kernel 
matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))
# --------------------------

# Check result with serial/CPU version
result_serial = np.matmul(A, B)
print("\nYour result is {}".format("correct!" if np.allclose(result_serial, C) else "incorrect."))

### ** It seems that higher threads-per-block (16**2=256 vs. 1) and lower block count (20000 vs. 5120000) performs better **

---------------------
# [Section 6] Matrix multiplication - Simple CUDA Solution + Shared CUDA Memory

In [ ]:
from numba import cuda, float32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
# TPB should not be larger than 32 in this example
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B using CUDA shared memory.

    Reference: https://stackoverflow.com/a/64198479/13697228 by @RobertCrovella
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

In [ ]:
# Controls threads per block and shared memory usage.

# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

#TODO - larger tiling - 256 tiling for 1024 matrix

# Initialize the data arrays
A = np.random.rand(TPB*100, TPB*100) # random matrix
B = np.random.rand(TPB*100, TPB*100) # random matrix
C = np.zeros((A.shape[0], B.shape[1])) # output matrix

# Configure the blocks
num_threads_per_block = (TPB, TPB)
blocks_x = int(math.ceil(A.shape[0] / num_threads_per_block[1]))
blocks_y = int(math.ceil(B.shape[1] / num_threads_per_block[0]))
num_blocks = (blocks_x, blocks_y)
print(f"num_blocks: {num_blocks}, num_threads_per_block: {num_threads_per_block}")

# JIT compilation/caching
fast_matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()

# --------------------------
C = np.zeros((A.shape[0], B.shape[1])) # output matrix
start_time = time.time()
print("\n--- [simple CUDA + shared memory] Starting the timer ---")
# Start the kernel 
fast_matmul[num_blocks, num_threads_per_block](A, B, C); cuda.synchronize()
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))
# --------------------------

# Check result with serial/CPU version
result_serial = np.matmul(A, B)
print("\nYour result is {}".format("correct!" if np.allclose(result_serial, C) else "incorrect."))